<a href="https://colab.research.google.com/github/maxmatical/Machine-Learning/blob/master/Yamana_Case_v4_Max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps taken in this notebook

1. use all features (excluding the highly correlated ones)
2. try LR and RF (tuned)
    - add feature importance
3. Try steps 1 and 2 on the dataset after dropping missing target values

In [0]:
# Set up environment and download course-v3
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai
!pip install fastprogress
!pip install pathlib

!curl https://course-v3.fast.ai/setup/colab | bash

In [0]:
!pip install scikit-optimize

In [0]:
from fastai import *
from fastai.tabular import *

import pandas as pd
import matplotlib as plt
import numpy as np

In [0]:
import matplotlib.pyplot as plt


# Using imputed values (median) for target variables

In [0]:
path = '/content/data/'
os.makedirs(path, exist_ok=True)
fname = 'Base de Datos Geometalurgia Goldspot'
#path

for some reason 



```
df = pd.read_csv(f'{path}/{name}.csv', low_memory=False)
```

is causing colab issues and cannot load


In [0]:
df = pd.read_csv('/content/data/Base de Datos Geometalurgia Goldspot.csv', low_memory=False)

In [0]:
df.head(5)

,compid,dhid,midx,midy,midz,topx,topy,topz,botx,boty,...,Te,Th,Ti,Tl,U,V,W,Y,Zn,Zr
0,CAN,OF07482,449967.09,7301211.35,1819.01,449967.61,7301211.40,1819.01,449966.56,7301211.31,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
1,CAN,OF07491,449950.70,7301374.94,1836.81,449949.32,7301375.18,1836.81,449952.08,7301374.70,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
2,CAN,OF07505,449950.06,7301418.56,1836.86,449949.61,7301418.52,1836.86,449950.50,7301418.60,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
3,CAN,OF08219,449896.24,7301571.17,1677.28,449895.20,7301571.08,1677.28,449897.29,7301571.26,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
4,CAN,OF08222,449906.02,7301433.32,1601.00,449905.37,7301433.32,1601.00,449906.67,7301433.32,...,-99,-99,24,-99,-99,-99.0,3,-99,1000,-99.0


### Preprocessing
1. remove any instances of ">", "<", and "," from the data
2. Impute (or remove) any instances of -99 for 'agrec' or 'nacnc'
    - if imputing: use median value 
3. Change any instance of -99 in features to be np.nan (to use FillMissing)
4. use FillMissing to change NaN to -99, and add feature for var_na (bool)
5. Perform analysis on features to see if any features (other than top/mid/bot) are correlated, remove those variables from the modelling process
    - https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
6. Split features into 3 dataframes: 
    - Just geo data
    - Just geochemical data
    - both
7. Model using features, see what kind of results we get with subset of the features
8. Perform any feature importance analysis as needed

In [0]:
# stripping all str columns of > and < symbols
for column in df.columns:
    if df[column].dtype == 'O' and column != 'compid' and column != 'dhid':
        df[column] = df[column].map(lambda x: x.lstrip('>,<'))
        df[column] = df[column].str.replace(',','.')
        df[column] = df[column].astype(float)



In [0]:
df['nacnc'] = df['nacnc'].replace(-99,df['nacnc'].median(), inplace = False)

df['agrec'] = df['agrec'].replace(-99,df['agrec'].median(), inplace = False)

In [0]:
df = df.replace(-99, np.nan, inplace=False)


In [0]:
cat_names = ['geocod', 'bound']
cont_names = ['midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']

In [0]:
fillmissing = FillMissing(cat_names = cat_names, cont_names = cont_names, fill_strategy='CONSTANT', fill_val = -99.0)


In [0]:
fillmissing(df)

In [13]:
df.head(10)

,compid,dhid,midx,midy,midz,topx,topy,topz,botx,boty,...,Te_na,Th_na,Ti_na,Tl_na,U_na,V_na,W_na,Y_na,Zn_na,Zr_na
0,CAN,OF07482,449967.09,7301211.35,1819.01,449967.61,7301211.40,1819.01,449966.56,7301211.31,...,True,True,True,True,True,True,True,True,True,True
1,CAN,OF07491,449950.70,7301374.94,1836.81,449949.32,7301375.18,1836.81,449952.08,7301374.70,...,True,True,True,True,True,True,True,True,True,True
2,CAN,OF07505,449950.06,7301418.56,1836.86,449949.61,7301418.52,1836.86,449950.50,7301418.60,...,True,True,True,True,True,True,True,True,True,True
3,CAN,OF08219,449896.24,7301571.17,1677.28,449895.20,7301571.08,1677.28,449897.29,7301571.26,...,True,True,True,True,True,True,True,True,True,True
4,CAN,OF08222,449906.02,7301433.32,1601.00,449905.37,7301433.32,1601.00,449906.67,7301433.32,...,True,True,False,True,True,True,False,True,False,True
5,CAN,OF08223,449940.31,7301543.05,1773.45,449940.01,7301543.08,1773.45,449940.61,7301543.03,...,True,True,True,True,True,True,True,True,True,True
6,CAN,OF08264,449922.79,7301652.09,1854.98,449921.08,7301651.73,1854.98,449924.50,7301652.46,...,True,True,False,True,True,True,False,True,False,True
7,CAN,OF08306,449933.71,7301556.51,1773.12,449933.33,7301556.19,1773.12,449934.10,7301556.83,...,True,True,False,True,True,True,False,True,False,True
8,CAN,OF08389,449872.89,7301556.36,1600.87,449871.73,7301556.05,1600.87,449874.05,7301556.67,...,True,True,False,True,True,True,False,True,False,True
9,CAN,OF09245,449873.89,7301509.13,1547.19,449873.25,7301508.99,1547.19,449874.52,7301509.26,...,True,True,True,True,True,True,True,True,True,True


In [0]:
cat_names = ['geocod', 'bound',
            'Al_na', 'As_na', 'Au_na', 'B_na', 'Ba_na', 'Be_na', 'Bi_na', 'Ca_na',
             'Cd_na', 'Co_na', 'Cr_na', 'Cu_na', 'Fe_na', 'Hg_na', 'K_na', 'La_na', 'Li_na', 'Mg_na',
             'Mn_na', 'Mo_na', 'Na_na',
             'Nb_na', 'Ni_na', 'P_na', 'Pb_na', 'Pd_na', 'Re_na', 'S_na', 'Sb_na', 'Se_na',
             'Si_na', 'Sc_na', 'Sn_na', 'Sr_na',
             'Ta_na', 'Te_na', 'Th_na', 'Ti_na', 'Tl_na', 'U_na', 'V_na', 'W_na', 'Y_na', 'Zn_na', 'Zr_na']

cont_names= ['midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']

Finding correlated features

In [0]:
df_orig_features = df[['geocod', 'bound', 'midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']]

In [0]:
corr_matrix = df_orig_features.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))



In [0]:
print(upper)

In [0]:
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(to_drop)


['bound', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz', 'Au', 'K', 'Pd', 'Re', 'Sn']


['bound', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz', 'Au', 'K', 'Pd', 'Re', 'Sn'] are all highly correlated with other features in the dataset


### Questions
- should we drop highly correlated chemical information?(Au, K, etc.)
- what sets of features does auppm and agppm fit in?
- can we include [vsed, tfil, dens] as features (i.e. can they be easily obtained and used to predict our target variables?)

In [0]:
df_all = df[['geocod','midx', 'midy', 'midz', 'length', 'from', 'to','auppm', 'agppm',
             'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg',
             'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se',
             'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr', 
             'Al_na', 'As_na', 'Au_na', 'B_na', 'Ba_na', 'Be_na', 'Bi_na', 'Ca_na',
             'Cd_na', 'Co_na', 'Cr_na', 'Cu_na', 'Fe_na', 'Hg_na', 'K_na', 'La_na', 'Li_na', 'Mg_na',
             'Mn_na', 'Mo_na', 'Na_na',
             'Nb_na', 'Ni_na', 'P_na', 'Pb_na', 'Pd_na', 'Re_na', 'S_na', 'Sb_na', 'Se_na',
             'Si_na', 'Sc_na', 'Sn_na', 'Sr_na',
             'Ta_na', 'Te_na', 'Th_na', 'Ti_na', 'Tl_na', 'U_na', 'V_na', 'W_na', 'Y_na', 'Zn_na', 'Zr_na']]

y_au = df['aurec']
y_ag = df['agrec']
y_nac = df['nacnc']
y_codt = df['codt']

In [0]:
# defining function to evaluate exp_rmse
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

def rmspe(y_pred, targ):
    pct_var = (targ - y_pred)/targ
    return math.sqrt((pct_var**2).mean())

In [0]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LogisticRegression, LinearRegression


In [0]:
n_features = df_all.shape[1]
from sklearn.model_selection import cross_val_score
from skopt.space import Real, Integer, Categorical 
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize


In [19]:
# list of valid scoring methods for objective function
from sklearn.metrics import SCORERS

sorted(SCORERS.keys()) 

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

## Predicting codt class

### Random Forest

using bayesian optimization to tune random forest

In [0]:
# Defining initial rf model
m = RandomForestClassifier(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space for RF
space = [Integer(100,2000, name='n_estimators'),
         Real(0.1, 1.0, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf'),
         Categorical(['gini', 'entropy'], name = 'criterion')]

In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return np.mean(cross_val_score(m, df_all, y_codt, cv=5, n_jobs=-1,
                                    scoring="accuracy"))

In [0]:
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [0]:
print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d
- criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4], res_gp.x[5]))

Best parameters:
- n_estimators=100
- max_features=0
- max_depth=1
- min_samples_split=2
- min_samples_leaf=100
- criterion = gini


tuning models doesn't seem to help 

Fitting model with the tuned hyperparameters

In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_all, y_codt, test_size = 0.2)

In [82]:
m = RandomForestClassifier(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

m.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.99, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [0]:
m.score(x_test, y_test)

0.9705882352941176

In [0]:
from sklearn.metrics import confusion_matrix

confusion_matrix(m.predict(x_test), y_test)

array([[ 98,   0,   1,   0],
       [  0,   1,   0,   1],
       [  1,   0,  31,   0],
       [  1,   4,   0, 134]])

In [0]:
from sklearn.metrics import classification_report

print(classification_report(m.predict(x_test), y_test))

              precision    recall  f1-score   support

           1       0.98      0.99      0.98        99
           2       0.20      0.50      0.29         2
           3       0.97      0.97      0.97        32
           4       0.99      0.96      0.98       139

   micro avg       0.97      0.97      0.97       272
   macro avg       0.79      0.86      0.80       272
weighted avg       0.98      0.97      0.97       272



In [83]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

        importance
midx      0.636016
midz      0.258002
midy      0.046552
geocod    0.026981
to        0.005482
from      0.003340
agppm     0.002808
auppm     0.002789
Au_na     0.002310
Au        0.002258
length    0.002168
As        0.001667
Zn        0.001211
Mg        0.001124
Cu        0.000872
Sb        0.000606
Ca        0.000522
Ti        0.000516
Si        0.000514
Mn        0.000418
Na        0.000393
S         0.000324
Te        0.000309
V         0.000300
K         0.000294
Zr        0.000280
Al        0.000267
Be        0.000241
Ba        0.000210
Cd        0.000180
...            ...
Zn_na     0.000000
Mn_na     0.000000
Ca_na     0.000000
Li_na     0.000000
U         0.000000
Hg        0.000000
Nb        0.000000
Re        0.000000
Se        0.000000
Sc        0.000000
Sn        0.000000
Sr        0.000000
Ta        0.000000
Bi        0.000000
Th        0.000000
Tl        0.000000
Y         0.000000
La_na     0.000000
B         0.000000
Al_na     0.000000
B_na      0.

### Logistic Regression

In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_all, y_codt, test_size = 0.2)

In [0]:
m = LogisticRegression()

m.fit(x_train, y_train)

In [0]:
m.score(x_test, y_test)

0.8345588235294118

In [0]:
confusion_matrix(m.predict(x_test), y_test)

array([[105,  35,   4],
       [  0,   1,   0],
       [  5,   1, 121]])

In [0]:
print(classification_report(m.predict(x_test), y_test))

              precision    recall  f1-score   support

           1       0.95      0.73      0.83       144
           3       0.03      1.00      0.05         1
           4       0.97      0.95      0.96       127

   micro avg       0.83      0.83      0.83       272
   macro avg       0.65      0.89      0.61       272
weighted avg       0.96      0.83      0.89       272



### Feature importance

- decision tree
- regegression p-value test
- random forest feature importance

In [0]:
f_regression(x_train, y_train)

(array([182.839135, 364.594091, 283.908306, 344.416313, ...,   7.784187,   2.053955,   8.07698 ,   1.966259]),
 array([1.282580e-38, 2.622899e-70, 9.042310e-57, 5.387003e-67, ..., 5.362773e-03, 1.520992e-01, 4.566997e-03,
        1.611313e-01]))

## Predicting aurec

### All info

In [0]:
# initial rf model
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return -np.mean(cross_val_score(m, df_all, y_au, cv=5, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [70]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=1404
- max_features=98
- max_depth=5
- min_samples_split=3
- min_samples_leaf=3


In [71]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=1404, max_features=98, min_samples_leaf=3, max_depth = 5,
                          min_samples_split = 3, n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

1.83986074502853 0.014214545056362682 0.11863280820694873


In [76]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

          importance
Au      3.131873e-01
auppm   2.304087e-01
agppm   1.501268e-01
midx    1.007796e-01
midy    4.323349e-02
midz    3.296875e-02
length  2.584065e-02
from    1.536959e-02
Fe      1.334880e-02
to      1.295521e-02
As      1.011605e-02
geocod  9.184072e-03
Cu      6.933313e-03
Te      4.964919e-03
Al      4.586643e-03
S       4.383775e-03
Ca      2.251669e-03
P       1.945394e-03
K       1.828499e-03
Mo      1.643639e-03
Ni      1.451604e-03
W       1.262401e-03
Cd      1.258218e-03
Na      1.057231e-03
Mg      9.809115e-04
Sb      8.202728e-04
Mn      7.103363e-04
Pb      6.835181e-04
Cr      6.212240e-04
Ta      5.587747e-04
...              ...
Si_na   1.667031e-05
Nb_na   1.659171e-05
Li      1.509992e-05
Cu_na   1.385502e-05
Mo_na   1.312147e-05
P_na    1.182222e-05
Ti_na   1.093124e-05
Zn_na   9.378307e-06
Bi_na   9.192515e-06
K_na    7.854523e-06
S_na    7.550282e-06
Tl      7.491754e-06
Mg_na   7.341172e-06
Ca_na   6.081311e-06
U       5.301532e-06
Pd_na   3.491

### Using linear regression

In [23]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = LinearRegression(n_jobs=-1)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

688.2234152896447 1628944.5061169674 598.357988980053


### Feature Importance

## Predicting agrec

### Random Forest

In [0]:
# initial rf model
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return -np.mean(cross_val_score(m, df_all, y_ag, cv=5, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [64]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=783
- max_features=46
- max_depth=5
- min_samples_split=16
- min_samples_leaf=1


In [77]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_ag, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=783, max_features=46, max_depth = 5,
                              min_samples_split = 16, min_samples_leaf=1,
                              n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))
    
    



3.236608936222217 0.027674334683116957 0.16608432393841577


In [78]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

        importance
midy      0.281090
midx      0.191005
agppm     0.115098
Au        0.091671
auppm     0.085522
midz      0.043657
Cu        0.033069
geocod    0.023188
from      0.013219
S         0.012587
to        0.012491
length    0.012249
As        0.009755
Pb        0.007756
Zn        0.007046
Cd        0.006836
Al        0.005087
K         0.004553
Ca        0.004308
Mn        0.004288
Fe        0.003841
Mg        0.003593
Bi        0.003486
Be        0.003172
Sb        0.003022
Na        0.002666
V         0.002165
Ni        0.001803
Mo        0.001441
P         0.001429
...            ...
Al_na     0.000033
Nb_na     0.000033
Sr_na     0.000027
Th        0.000026
Pd_na     0.000017
Sn_na     0.000017
U_na      0.000016
Y_na      0.000013
Ta        0.000012
Y         0.000010
Mn_na     0.000008
Ta_na     0.000008
W_na      0.000006
Sc        0.000005
Pb_na     0.000005
Cd_na     0.000002
V_na      0.000000
Th_na     0.000000
As_na     0.000000
Sc_na     0.000000
Sn        0.

### Linear Regression

In [34]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = LinearRegression(n_jobs=-1)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

195.07249448783168 215372.0092706498 168.10968404431244


### Feature Importance

## Predicting nacnc

### All info

In [0]:
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
# @use_named_args(space)
# def objective(**params):
#     m.set_params(**params)

#     return -np.mean(cross_val_score(m, df_all, y_nac, cv=5, n_jobs=-1,
#                                     scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [50]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=71
- max_features=76
- max_depth=5
- min_samples_split=14
- min_samples_leaf=3


In [79]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=71, max_features=70, max_depth = 5,
                              min_samples_split = 14, min_samples_leaf=3,
                          n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

5.734362323648336 2.575465520999889 1.6033991310665132


In [80]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

        importance
midy      0.253399
midx      0.235121
agppm     0.127003
midz      0.081699
Au        0.055066
auppm     0.046668
Si        0.034176
length    0.026882
from      0.023109
geocod    0.020326
Cr        0.013453
to        0.012482
Pd        0.007295
Cu        0.006022
Sb        0.005548
Hg        0.005322
Bi        0.005180
Ti        0.004125
Ni        0.004111
Au_na     0.003805
Zr        0.003282
Na        0.003246
Fe        0.002299
Mo        0.002115
V         0.001753
W         0.001723
Pb        0.001640
Zn        0.001440
Mn        0.001121
Ca        0.001088
...            ...
Ta_na     0.000000
Mn_na     0.000000
Mo_na     0.000000
Cd_na     0.000000
Mg_na     0.000000
La_na     0.000000
La        0.000000
Nb        0.000000
Co        0.000000
Re        0.000000
Se        0.000000
Sc        0.000000
Sn        0.000000
Ta        0.000000
Th        0.000000
Ba        0.000000
Tl        0.000000
U         0.000000
B         0.000000
Y         0.000000
As_na     0.

## Observations



#Dropping missing values for target variables

In [0]:
path = '/content/data/'
os.makedirs(path, exist_ok=True)
fname = 'Base de Datos Geometalurgia Goldspot'
#path

for some reason 



```
df = pd.read_csv(f'{path}/{name}.csv', low_memory=False)
```

is causing colab issues and cannot load


In [0]:
df = pd.read_csv('/content/data/Base de Datos Geometalurgia Goldspot.csv', low_memory=False)

In [86]:
df.head(5)

,compid,dhid,midx,midy,midz,topx,topy,topz,botx,boty,...,Te,Th,Ti,Tl,U,V,W,Y,Zn,Zr
0,CAN,OF07482,449967.09,7301211.35,1819.01,449967.61,7301211.40,1819.01,449966.56,7301211.31,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
1,CAN,OF07491,449950.70,7301374.94,1836.81,449949.32,7301375.18,1836.81,449952.08,7301374.70,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
2,CAN,OF07505,449950.06,7301418.56,1836.86,449949.61,7301418.52,1836.86,449950.50,7301418.60,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
3,CAN,OF08219,449896.24,7301571.17,1677.28,449895.20,7301571.08,1677.28,449897.29,7301571.26,...,-99,-99,-99,-99,-99,-99.0,-99,-99,-99,-99.0
4,CAN,OF08222,449906.02,7301433.32,1601.00,449905.37,7301433.32,1601.00,449906.67,7301433.32,...,-99,-99,24,-99,-99,-99.0,3,-99,1000,-99.0


### Preprocessing
1. remove any instances of ">", "<", and "," from the data
2. Impute (or remove) any instances of -99 for 'agrec' or 'nacnc'
    - if imputing: use median value 
3. Change any instance of -99 in features to be np.nan (to use FillMissing)
4. use FillMissing to change NaN to -99, and add feature for var_na (bool)
5. Perform analysis on features to see if any features (other than top/mid/bot) are correlated, remove those variables from the modelling process
    - https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
6. Split features into 3 dataframes: 
    - Just geo data
    - Just geochemical data
    - both
7. Model using features, see what kind of results we get with subset of the features
8. Perform any feature importance analysis as needed

In [0]:
# stripping all str columns of > and < symbols
for column in df.columns:
    if df[column].dtype == 'O' and column != 'compid' and column != 'dhid':
        df[column] = df[column].map(lambda x: x.lstrip('>,<'))
        df[column] = df[column].str.replace(',','.')
        df[column] = df[column].astype(float)



In [0]:
# dropping -99 values in target variables
df = df[df['agrec']!=-99]
df = df[df['nacnc']!=-99]

In [0]:
df = df.replace(-99, np.nan, inplace=False)


In [0]:
cat_names = ['geocod', 'bound']
cont_names = ['midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']

In [0]:
fillmissing = FillMissing(cat_names = cat_names, cont_names = cont_names, fill_strategy='CONSTANT', fill_val = -99.0)


In [0]:
fillmissing(df)

In [102]:
df.head(10)

,compid,dhid,midx,midy,midz,topx,topy,topz,botx,boty,...,Te_na,Th_na,Ti_na,Tl_na,U_na,V_na,W_na,Y_na,Zn_na,Zr_na
0,CAN,OF07482,449967.09,7301211.35,1819.01,449967.61,7301211.40,1819.01,449966.56,7301211.31,...,True,True,True,True,True,True,True,True,True,True
1,CAN,OF07491,449950.70,7301374.94,1836.81,449949.32,7301375.18,1836.81,449952.08,7301374.70,...,True,True,True,True,True,True,True,True,True,True
2,CAN,OF07505,449950.06,7301418.56,1836.86,449949.61,7301418.52,1836.86,449950.50,7301418.60,...,True,True,True,True,True,True,True,True,True,True
3,CAN,OF08219,449896.24,7301571.17,1677.28,449895.20,7301571.08,1677.28,449897.29,7301571.26,...,True,True,True,True,True,True,True,True,True,True
4,CAN,OF08222,449906.02,7301433.32,1601.00,449905.37,7301433.32,1601.00,449906.67,7301433.32,...,True,True,False,True,True,True,False,True,False,True
5,CAN,OF08223,449940.31,7301543.05,1773.45,449940.01,7301543.08,1773.45,449940.61,7301543.03,...,True,True,True,True,True,True,True,True,True,True
6,CAN,OF08264,449922.79,7301652.09,1854.98,449921.08,7301651.73,1854.98,449924.50,7301652.46,...,True,True,False,True,True,True,False,True,False,True
7,CAN,OF08306,449933.71,7301556.51,1773.12,449933.33,7301556.19,1773.12,449934.10,7301556.83,...,True,True,False,True,True,True,False,True,False,True
8,CAN,OF08389,449872.89,7301556.36,1600.87,449871.73,7301556.05,1600.87,449874.05,7301556.67,...,True,True,False,True,True,True,False,True,False,True
9,CAN,OF09245,449873.89,7301509.13,1547.19,449873.25,7301508.99,1547.19,449874.52,7301509.26,...,True,True,True,True,True,True,True,True,True,True


In [0]:
cat_names = ['geocod', 'bound',
            'Al_na', 'As_na', 'Au_na', 'B_na', 'Ba_na', 'Be_na', 'Bi_na', 'Ca_na',
             'Cd_na', 'Co_na', 'Cr_na', 'Cu_na', 'Fe_na', 'Hg_na', 'K_na', 'La_na', 'Li_na', 'Mg_na',
             'Mn_na', 'Mo_na', 'Na_na',
             'Nb_na', 'Ni_na', 'P_na', 'Pb_na', 'Pd_na', 'Re_na', 'S_na', 'Sb_na', 'Se_na',
             'Si_na', 'Sc_na', 'Sn_na', 'Sr_na',
             'Ta_na', 'Te_na', 'Th_na', 'Ti_na', 'Tl_na', 'U_na', 'V_na', 'W_na', 'Y_na', 'Zn_na', 'Zr_na']

cont_names= ['midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']

Finding correlated features

In [0]:
df_orig_features = df[['geocod', 'bound', 'midx', 'midy', 'midz', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz',
             'length', 'from', 'to','auppm', 'agppm', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg', 'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se', 'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr']]

In [0]:
corr_matrix = df_orig_features.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))



In [0]:
print(upper)

In [107]:
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(to_drop)


['bound', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz', 'Au', 'K', 'Pd', 'Re', 'Sn']


['bound', 'topx', 'topy', 'topz', 'botx', 'boty', 'botz', 'Au', 'K', 'Pd', 'Re', 'Sn'] are all highly correlated with other features in the dataset


### Questions
- should we drop highly correlated chemical information?(Au, K, etc.)
- what sets of features does auppm and agppm fit in?
- can we include [vsed, tfil, dens] as features (i.e. can they be easily obtained and used to predict our target variables?)

In [0]:
df_all = df[['geocod','midx', 'midy', 'midz', 'length', 'from', 'to','auppm', 'agppm',
             'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Ca',
             'Cd', 'Co', 'Cr', 'Cu', 'Fe', 'Hg', 'K', 'La', 'Li', 'Mg',
             'Mn', 'Mo', 'Na',
             'Nb', 'Ni', 'P', 'Pb', 'Pd', 'Re', 'S', 'Sb', 'Se',
             'Si', 'Sc', 'Sn', 'Sr',
             'Ta', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Y', 'Zn', 'Zr', 
             'Al_na', 'As_na', 'Au_na', 'B_na', 'Ba_na', 'Be_na', 'Bi_na', 'Ca_na',
             'Cd_na', 'Co_na', 'Cr_na', 'Cu_na', 'Fe_na', 'Hg_na', 'K_na', 'La_na', 'Li_na', 'Mg_na',
             'Mn_na', 'Mo_na', 'Na_na',
             'Nb_na', 'Ni_na', 'P_na', 'Pb_na', 'Pd_na', 'Re_na', 'S_na', 'Sb_na', 'Se_na',
             'Si_na', 'Sc_na', 'Sn_na', 'Sr_na',
             'Ta_na', 'Te_na', 'Th_na', 'Ti_na', 'Tl_na', 'U_na', 'V_na', 'W_na', 'Y_na', 'Zn_na', 'Zr_na']]

y_au = df['aurec']
y_ag = df['agrec']
y_nac = df['nacnc']
y_codt = df['codt']

In [0]:
# defining function to evaluate exp_rmse
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

def rmspe(y_pred, targ):
    pct_var = (targ - y_pred)/targ
    return math.sqrt((pct_var**2).mean())

In [0]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LogisticRegression, LinearRegression


In [0]:
n_features = df_all.shape[1]
from sklearn.model_selection import cross_val_score
from skopt.space import Real, Integer, Categorical 
from skopt.utils import use_named_args
from skopt import gp_minimize, forest_minimize, gbrt_minimize


In [16]:
# list of valid scoring methods for objective function
from sklearn.metrics import SCORERS

sorted(SCORERS.keys()) 

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

## Predicting codt class

### Random Forest

using bayesian optimization to tune random forest

In [0]:
# Defining initial rf model
m = RandomForestClassifier(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space for RF
space = [Integer(100,2000, name='n_estimators'),
         Real(0.1, 1.0, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf'),
         Categorical(['gini', 'entropy'], name = 'criterion')]

In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return np.mean(cross_val_score(m, df_all, y_codt, cv=5, n_jobs=-1,
                                    scoring="accuracy"))

In [0]:
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [0]:
print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d
- criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4], res_gp.x[5]))

Best parameters:
- n_estimators=100
- max_features=0
- max_depth=1
- min_samples_split=2
- min_samples_leaf=100
- criterion = gini


tuning models doesn't seem to help 

Fitting model with the tuned hyperparameters

In [0]:
x_train, x_test, y_train, y_test = train_test_split(df_all, y_codt, test_size = 0.2)

In [20]:
m = RandomForestClassifier(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

m.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.99, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [21]:
m.score(x_test, y_test)

0.9695817490494296

In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix(m.predict(x_test), y_test)

array([[102,   0,   2,   1],
       [  0,   1,   0,   0],
       [  3,   0,  32,   0],
       [  2,   0,   0, 120]])

In [23]:
from sklearn.metrics import classification_report

print(classification_report(m.predict(x_test), y_test))

              precision    recall  f1-score   support

           1       0.95      0.97      0.96       105
           2       1.00      1.00      1.00         1
           3       0.94      0.91      0.93        35
           4       0.99      0.98      0.99       122

   micro avg       0.97      0.97      0.97       263
   macro avg       0.97      0.97      0.97       263
weighted avg       0.97      0.97      0.97       263



In [25]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances[:5])

        importance
midx      0.651735
midz      0.250366
midy      0.043041
geocod    0.022391
to        0.004532


In [27]:
importances = m.feature_importances_
std = np.std([m.feature_importances_ for tree in m.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]


Feature ranking:


In [0]:

# # Plot the feature importances of the forest
# plt.figure()
# plt.title("Feature importances for co")
# plt.bar( importances[indices], range(x_train.shape[1]),
#        color="r", yerr=std[indices], align="center")
# plt.xticks(range(x_train.shape[1]), indices)
# plt.xlim([-1, x_train.shape[1]])
# plt.show()

## Predicting aurec

### All info

In [0]:
# initial rf model
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return -np.mean(cross_val_score(m, df_all, y_au, cv=5, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [125]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=2000
- max_features=79
- max_depth=5
- min_samples_split=100
- min_samples_leaf=1


In [126]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=2000, max_features=79, min_samples_leaf=1, max_depth = 5,
                          min_samples_split = 100, n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

1.0789921030057927 0.014056742342823633 0.11748708970457952


In [127]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

        importance
Au        0.348066
auppm     0.299779
agppm     0.134504
midx      0.120734
midy      0.041039
Te        0.010087
length    0.007132
midz      0.004699
from      0.004406
Al        0.004195
K         0.003618
Cu        0.003440
Mo        0.002328
Ca        0.001719
As        0.001664
S         0.001375
Sb        0.001251
W         0.000959
Mn        0.000897
Cr_na     0.000840
P         0.000800
Mg        0.000740
to        0.000720
W_na      0.000668
Fe        0.000635
geocod    0.000609
Zn        0.000403
Ba        0.000373
Si        0.000329
Pb        0.000269
...            ...
V_na      0.000000
B         0.000000
Zn_na     0.000000
Nb_na     0.000000
Hg_na     0.000000
Mo_na     0.000000
Be_na     0.000000
Se        0.000000
Nb        0.000000
Tl        0.000000
U         0.000000
Hg        0.000000
Zr        0.000000
Al_na     0.000000
As_na     0.000000
B_na      0.000000
Ba_na     0.000000
Bi_na     0.000000
Mn_na     0.000000
Ca_na     0.000000
Cd_na     0.

### Using linear regression

In [0]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = LinearRegression(n_jobs=-1)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

688.2234152896447 1628944.5061169674 598.357988980053


### Feature Importance

## Predicting agrec

### Random Forest

In [0]:
# initial rf model
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
@use_named_args(space)
def objective(**params):
    m.set_params(**params)

    return -np.mean(cross_val_score(m, df_all, y_ag, cv=5, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [133]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=2000
- max_features=47
- max_depth=5
- min_samples_split=2
- min_samples_leaf=1


In [136]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_ag, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=2000, max_features=47, max_depth = 5,
                              min_samples_split = 2, min_samples_leaf=1,
                              n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))
    
    



3.76160750332823 0.02845393680847842 0.16845909087299007


In [135]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

          importance
midy    3.060188e-01
midx    1.727030e-01
agppm   1.184096e-01
Au      7.775442e-02
auppm   7.439559e-02
midz    5.211661e-02
Cu      3.682217e-02
geocod  1.828005e-02
length  1.442013e-02
from    1.224289e-02
Mn      1.107335e-02
to      1.104189e-02
As      9.576248e-03
Sb      8.167864e-03
Cd      7.415221e-03
Al      6.466568e-03
Zn      5.340140e-03
Be      5.145300e-03
K       4.977417e-03
Pb      4.903730e-03
Fe      4.662726e-03
Mg      4.660834e-03
Ca      4.076790e-03
Na      3.733249e-03
S       3.027758e-03
Ni      2.346909e-03
Bi      2.282629e-03
P       2.182621e-03
Mo      2.111129e-03
Cr      1.282793e-03
...              ...
Th      4.785517e-05
S_na    4.418905e-05
Ni_na   4.211693e-05
Pb_na   4.207387e-05
Ti_na   3.978728e-05
Tl_na   3.932443e-05
Nb_na   3.901649e-05
Re_na   3.723083e-05
Sb_na   3.655410e-05
Bi_na   3.476936e-05
Zr_na   3.457709e-05
V_na    2.834489e-05
Sn      2.833296e-05
Na_na   2.750926e-05
Sr_na   2.621490e-05
Ca_na   2.265

### Linear Regression

In [0]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_au, test_size = 0.2)
    m = LinearRegression(n_jobs=-1)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

195.07249448783168 215372.0092706498 168.10968404431244


### Feature Importance

## Predicting nacnc

### All info

In [0]:
m = RandomForestRegressor(n_estimators=100, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
# defining search space
space = [Integer(10,2000, name='n_estimators'),
         Integer(1, n_features, name='max_features'),
         Integer(1, 5, name='max_depth'),
         Integer(2, 100, name='min_samples_split'),
         Integer(1, 100, name='min_samples_leaf')]


In [0]:
# estimator parameters
# @use_named_args(space)
# def objective(**params):
#     m.set_params(**params)

#     return -np.mean(cross_val_score(m, df_all, y_nac, cv=5, n_jobs=-1,
#                                     scoring="neg_mean_squared_error"))

In [0]:
# trying a different objective function
@use_named_args(space)
def objective(**params):
    m.set_params(**params)
    i = 0
    l_rmse=[]
    while i<10:
        x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
        m.fit(x_train, y_train)
        preds = m.predict(x_test)
        l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
        i+=1

    return np.mean(l_rmse)

In [0]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)
#instead of gp_minimize can also use forest_minimize or gbrt_minimize

%time res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)


"Best score=%.4f" % res_gp.fun

In [142]:
# print("""Best parameters:
# - n_estimators=%d
# - max_features=%d
# - max_depth=%d
# - min_samples_split=%d
# - min_samples_leaf=%d
# - criterion = %s""" % (res_gp.x[0], res_gp.x[1], 
#                             res_gp.x[2], res_gp.x[3], 
#                             res_gp.x[4], res_gp.x[5]))

print("""Best parameters:
- n_estimators=%d
- max_features=%d
- max_depth=%d
- min_samples_split=%d
- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1], 
                            res_gp.x[2], res_gp.x[3], 
                            res_gp.x[4]))

Best parameters:
- n_estimators=2000
- max_features=57
- max_depth=5
- min_samples_split=2
- min_samples_leaf=1


In [145]:
i = 0
l_rmspe = []
l_mse = []
l_rmse=[]

while i <10:
    x_train, x_test, y_train, y_test = train_test_split(df_all, y_nac, test_size = 0.2)
    m = RandomForestRegressor(n_estimators=2000, max_features=57, max_depth = 5,
                              min_samples_split = 2, min_samples_leaf=1,
                          n_jobs=-1, oob_score=True)
    m.fit(x_train, y_train)
    preds = m.predict(x_test)
    l_rmspe.append(rmspe(preds, y_test))
    l_mse.append(mean_squared_error(preds, y_test))
    l_rmse.append(math.sqrt(mean_squared_error(preds, y_test)))
    i+=1

print(np.mean(l_rmspe), np.mean(l_mse), np.mean(l_rmse))

4.134852036876245 2.870554495073887 1.6927921587353747


In [144]:
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

          importance
midx    2.075027e-01
midy    2.002600e-01
midz    9.953809e-02
agppm   8.999698e-02
auppm   8.754112e-02
Au      8.740838e-02
Si      4.401867e-02
Cr      2.835189e-02
Sb      2.026161e-02
length  1.929463e-02
geocod  1.722855e-02
from    1.597678e-02
to      1.278347e-02
Cu      5.855616e-03
Ti      3.854497e-03
Na      3.825642e-03
Hg      3.352738e-03
Bi      3.187677e-03
V       3.074214e-03
Pd      2.512209e-03
Ni      2.412585e-03
Zr_na   2.361689e-03
Pb      2.253683e-03
Sr_na   2.075198e-03
Ba_na   2.031136e-03
Tl_na   1.970587e-03
La_na   1.946017e-03
Te_na   1.877490e-03
V_na    1.829781e-03
S       1.787789e-03
...              ...
Mn_na   4.051975e-05
Fe_na   3.823522e-05
Bi_na   3.791566e-05
Ta      3.642196e-05
Tl      3.617716e-05
Be_na   3.464187e-05
Sc_na   3.387120e-05
Hg_na   3.209409e-05
Sc      3.172204e-05
Th      2.827392e-05
S_na    2.759931e-05
Li_na   2.750368e-05
Mo_na   2.646930e-05
Si_na   2.574035e-05
Ti_na   2.447410e-05
Ni_na   2.423

## Observations

- Dropping observations where the target variable was missing (eg agrec, nacnc) did not yield any increase in predictive power

